# 4304 Project Infographic

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import geopandas as gpd
from mpl_toolkits.basemap import Basemap
from geopy.geocoders import Nominatim
import time

___________________

In [2]:
df = pd.read_csv('satcat.tsv', sep='\t', low_memory=False)

In [3]:
orgs = pd.read_csv("orgs.tsv", sep="\t", usecols=["#Code", "Class", "ShortName"]).drop(0)
orgs.rename(columns={"#Code": "Owner"}, inplace=True)

In [4]:
class_mapping = {
    "A": "Academic/Non-Profit",
    "B": "Business/Commercial",
    "C": "Civil Government",
    "D": "Defense/Military"
}

orbit_mapping = {
    'LEO': ['LEO/I', 'LLEO/I', 'LEO/P', 'LLEO/P', 'LEO/S', 'LEO/E', 'LEO/R', 'LLEO/R', 'LLEO/S', 'LLEO/E'],
    'MEO': ['MEO'],
    'GEO': ['GEO/NS', 'GEO/T', 'GEO/S', 'GEO/D', 'GEO/I', 'GEO/ID'],
    'GTO': ['GTO'],
    'HEO': ['HEO', 'VHEO', 'HEO/M'],
    'Deep Space': ['DSO', 'SO', 'CLO', 'TA'],
    'Unclassified': ['-']
}

def get_class_for_owner(owner):
    owners = owner.split("/")
    classes = orgs[orgs["Owner"].isin(owners)]["Class"].unique()
    
    return classes[0] if len(classes) > 0 else None
    
def map_orbit_type(oporbit):
    for category, types in orbit_mapping.items():
        if oporbit in types:
            return category
    return 'Other'

df['OrbitType'] = df['OpOrbit'].apply(map_orbit_type)
df["Class"] = df["Owner"].apply(get_class_for_owner)
df = df[df['Status'] == 'O']

In [5]:
df = df.merge(orgs[['Owner', 'ShortName']], on='Owner', how='left')

__________________________

## Satellite bubble map

In [2]:
country_counts = df['State'].value_counts().reset_index()
country_counts.columns = ['Country', 'Satellite_Count']

# Approximate country coordinates (You can refine this with a geolocation dataset)
country_coords = {
    'US': (-98.35, 39.50),   # USA
    'RU': (105.32, 61.52),   # Russia
    'CN': (104.19, 35.86),   # China
    'IN': (78.96, 20.59),    # India
    'EU': (10.45, 51.17),    # Europe (approximate)
    'JP': (138.25, 36.20),   # Japan
    'UK': (-3.43, 55.38),    # United Kingdom
    'FR': (2.21, 46.23),     # France
    'DE': (10.45, 51.17),    # Germany
    'BR': (-51.93, -14.23),  # Brazil
}

# Merge country counts with coordinates
country_counts['Coordinates'] = country_counts['Country'].map(country_coords)
country_counts = country_counts.dropna()  # Remove missing locations

# Extract lat/lon
lons, lats, sizes = [], [], []
for _, row in country_counts.iterrows():
    lon, lat = row['Coordinates']
    lats.append(lat)
    lons.append(lon)
    sizes.append(row['Satellite_Count'])  # Bubble size is satellite count

# Normalize bubble sizes for better visualization
sizes = [s for s in sizes]  # Adjust multiplier for clarity

# Plot world map with bubbles
plt.figure(figsize=(12, 6))
m = Basemap(projection='robin', lon_0=0, resolution='c')

m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='lightgray', lake_color='lightblue')
m.drawmapboundary(fill_color='lightblue')

# Convert lat/lon to map coordinates
x, y = m(lons, lats)

m.scatter(x, y, s=sizes, color='red', alpha=0.6, edgecolors='k', linewidth=0.5)

# Title and legend
plt.title("Global Satellite Distribution - Bubble Map")
plt.show()

NameError: name 'df' is not defined